In [316]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import title_is
from selenium.webdriver.support import expected_conditions as EC
import bs4
import time
import re

In [490]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Firefox()

driver.get('https://www.jobth.com/searchjob2.php?typejob=&city=&province=&keyword=')

wait = WebDriverWait(driver, 10)

original_window = driver.current_window_handle

assert len(driver.window_handles) == 1

driver.execute_script("document.body.style.transform='scale(0.1)';")
driver.execute_script("document.body.style.transformOrigin='0 0';")

time.sleep(2)

driver.find_element(By.XPATH,'/html/body/center/div[3]/center/div/div[4]/a[4]').click()

wait.until(EC.number_of_windows_to_be(2))

for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            break

time.sleep(6)

data = driver.page_source
soup = bs4.BeautifulSoup(data)

listrole = []
listprovince = []
listsalary_rate = []
listjob_type = []
listrequire = []
listjob_details = []
listjobqualifications = []

P = soup.find_all("div",{"class":"w3-container w3-xlarge w3-center w3-theme-d5 w3-padding w3-text-shadow"}) 
for c0 in P:
    c0 = c0.text
    listrole.append(c0)

P = soup.find_all("div",{"class":"w3-container w3-left-align w3-medium w3-theme-l4"}) 
for c1 in P:
    c1 = c1.text
    match = re.search(r'จังหวัด (.+?) \(', c1)
    province = match.group(1) if match else None

    match = re.search(r'อัตราค่าจ้าง (.+?)\รูปแบบงาน', c1)
    salary_rate = match.group(1) if match else None

    match = re.search(r'รูปแบบงาน (.+?)\อัตรา', c1)
    job_type = match.group(1) if match else None

    match = re.search(r'อัตรา (.+?)\อัตรา', c1)
    require = match.group(1) if match else None

    listprovince.append(province)
    listsalary_rate.append(salary_rate)
    listjob_type.append(job_type)
    listrequire.append(require)

P = soup.find_all("div",{"class":"w3-container w3-left-align w3-medium w3-theme-l5"}) 

for c2 in P:
    c2 = c2.text
    job_details_pattern = re.compile(r'รายละเอียดงาน(.+?)คุณสมบัติผู้สมัคร', re.DOTALL)
    job_details_match = job_details_pattern.search(c2)
    if job_details_match:
        job_details = job_details_match.group(1)
        
    
    qualifications_pattern = re.compile(r'คุณสมบัติผู้สมัคร(.+?)วุฒิการศึกษาระดับ', re.DOTALL)
    qualifications_match = qualifications_pattern.search(c2)
    if qualifications_match:
        qualifications = qualifications_match.group(1).strip()

    listjob_details.append(job_details)
    listjobqualifications.append(qualifications)

df = pd.DataFrame()
# df['JobID'] = ProductID
df['RoleName'] = listrole
df['Province'] = listprovince
df['Salary_rate'] = listsalary_rate
df['Job_type'] = listjob_type
df['Require'] = listrequire
df['Job_details'] = listjob_details
df['jobqualifications'] = listjobqualifications

# driver.quit()

In [492]:
df

,RoleName,Province,Salary_rate,Job_type,Require,Job_details,jobqualifications
0,บัญชี/การเงิน,นนทบุรี,"16,000-22,000",งานประจำ (Full Time),None,ทำบัญชีบริษัท,มีประสบการณ์การทำงาน 1- 2 ปี มีไหวพริบ มีการแ...


In [482]:
P = soup.find_all("div",{"class":"w3-container w3-xlarge w3-center w3-theme-d5 w3-padding w3-text-shadow"}) 

for c0 in P:
    c0 = c0.text
    
    

โฟร์แมน โซนภาคเหนือ


In [496]:
P = soup.find_all("div",{"class":"w3-container w3-left-align w3-medium w3-theme-l4"}) 
P

for r0 in P:
    r0 = r0.text
    match = re.search(r'จังหวัด (.+?) \(', r0)
    province = match.group(1) if match else None

    match = re.search(r'อัตราค่าจ้าง (.+?)\รูปแบบงาน', r0)
    salary_rate = match.group(1) if match else None

    match = re.search(r'รูปแบบงาน (.+?)\อัตรา', r0)
    job_type = match.group(1) if match else None

    match = re.search(r'อัตรา (.+?)\อัตรา', r0)
    require = match.group(1) if match else 0
    num_positions = match.group(2) if match else 0
    

    print("จังหวัด:", province)
    print("อัตราค่าจ้าง:", salary_rate)
    print("รูปแบบงาน:", job_type)
    print("จำนวนตำแหน่งที่ต้องการ:", require)

จังหวัด: นนทบุรี
อัตราค่าจ้าง: 16,000-22,000
รูปแบบงาน: งานประจำ (Full Time)
จำนวนตำแหน่งที่ต้องการ: 0


In [458]:
P = soup.find_all("div",{"class":"w3-container w3-left-align w3-medium w3-theme-l5"}) 

for r0 in P:
    r0 = r0.text
    print(r0)

รายละเอียดงานดูแลควบคุมงานก่อสร้างช่วยวิศวกรหลักโครงการ
บริหารจัดการงานก่อสร้างเป็นไปตามแผนงาน
ติดต่อประสานงาน- เอกกสาร อื่นๆคุณสมบัติผู้สมัครมีประสบการณ์การทำงาน 1- 2 ปี  มีไหวพริบ มีการแก้ไขปัญหาได้ดี มีความขยัน อดทน รับแรงกดดันในการทำงานได้ ไม่ติดขัดในการเดินทางต่างจังหวัด ทำงานได้ทุกจังหวัด วุฒิการศึกษาระดับ ปวช. ขึ้นไปในสาขางานที่เกี่ยวข้อง ขับรถได้ มีใบขับขี่ มีรถส่วนตัว จะพิจารณาพิเศษ ทำเอกสารเบื้องต้นได้ ใช้คอมเป็น สวัสดิการOT
ที่พัก
ชุดฟอร์มบริษัท
โบนัสประจำปี
ประกันสังคมตามกฎหมาย
งานเลี้ยงสังสรรค์เทศกาลต่างๆสถานที่ปฏิบัติงานโครงการตามจังหวัดต่างๆวิธีการรับสมัครงานสมัครด้วยตนเอง
ส่งใบสมัครผ่านทาง email บริษัท : skp@sinkaew.com
ส่งใบสมัครผ่าน fax บริษัท
เบอร์ติดต่อ 094-0969587สวัสดิการ1.เข้าระบบประกันสังคม
2.มีโบนัสประจำปี / ปรับเงินประจำปี
3.ชุดฟอร์มฟรี
4.มีค่าที่พักให้เดือนละ 3000++
5.มีค่าเสื่อมรถส่วนตัวให้ เดือนละ 3000++
6.รถที่ใช้ในหน้างานสามารถเบิกค่าน้ำมันได้
7.เลี้ยงประจำปี / ประจำเดือน /ท่องเที่ยวภายใน-ภายนอก ประเทศ


In [498]:
P = soup.find_all("div",{"class":"w3-container w3-left-align w3-medium w3-theme-l5"}) 

for r0 in P:
    r0 = r0.text
    job_details_pattern = re.compile(r'รายละเอียดงาน(.+?)คุณสมบัติผู้สมัคร', re.DOTALL)
    job_details_match = job_details_pattern.search(r0)
    if job_details_match:
        job_details = job_details_match.group(1)
        
    
    qualifications_pattern = re.compile(r'คุณสมบัติผู้สมัคร(.+?)วุฒิการศึกษาระดับ', re.DOTALL)
    qualifications_match = qualifications_pattern.search(r0)
    if qualifications_match:
        qualifications = qualifications_match.group(1).strip()
        print("Qualifications:", qualifications)
        
    print("รายละเอียดงาน:", job_details)
    print("คุณสมบัติผู้สมัคร:", qualifications)

รายละเอียดงาน: ทำบัญชีบริษัท
คุณสมบัติผู้สมัคร: มีประสบการณ์การทำงาน 1- 2 ปี  มีไหวพริบ มีการแก้ไขปัญหาได้ดี มีความขยัน อดทน รับแรงกดดันในการทำงานได้ ไม่ติดขัดในการเดินทางต่างจังหวัด ทำงานได้ทุกจังหวัด


In [480]:
job_title = soup.find('div', class_='w3-container w3-xlarge w3-center w3-theme-d5 w3-padding w3-text-shadow').text
location = soup.find('p', class_='w3-medium').text
salary = soup.find('p', class_='w3-medium').find_next('p', class_='w3-medium').text

# Print the extracted information
print(f"Job Title: {job_title}")
print(f"Location: {location}")
print(f"Salary: {salary}")


Job Title: โฟร์แมน โซนภาคเหนือ
Location: จังหวัด พิษณุโลก (เมืองพิษณุโลก)
Salary: อัตราค่าจ้าง 12,000-18,000


In [ ]:
All_P = []
data = driver.page_source
soup = bs4.BeautifulSoup(data)
products = soup.select(".row>div")
for product in products:
    E = product.select_one(".scroll > div > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > h4:nth-child(1)").text.strip()
    D = product.select_one(".scroll > div > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > a:nth-child(2)").text.strip()
    A = product.select_one("div.job-description").text.strip()
    P = product.select_one("div.employment > b:nth-child(2)").text.strip()
    R = product.select_one("div.employment > b:nth-child(5)").text.strip()
    O = product.select_one("div.qualifications > span:nth-child(4)").text.strip()
    J = product.select_one("div.qualifications > span:nth-child(7)").text.strip()
    E = product.select_one("div.qualifications > span:nth-child(10)").text.strip()
    C = product.select_one("div.job-benefit > span:nth-child(3)").text.strip()
    T = product.select_one("div.contact-cpn > span:nth-child(3)").text.strip()

    All_P.append([E,D,A,P,R,O,J,E,C,T])
All_P